In [1]:
import set_url
import open_chrome
import columns

import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

import os
import time
import requests

In [2]:
dir = 'takamatsu/racepage/target/'

In [3]:
URL = "https://race.netkeiba.com/race/shutuba.html?race_id=202307020611&rf=race_list"#race_listのページを指定

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver, 3)
driver.get(URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

C:\Users\kawam\AppData\Local\Temp\ipykernel_30208\3173249733.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [5]:
HTML_DIR = "html/"+dir
if not os.path.isdir(HTML_DIR):
    os.makedirs(HTML_DIR)

url = driver.current_url

save_file_path = HTML_DIR+"/"+'target.html'
response = requests.get(url)
response.encoding = response.apparent_encoding
html = response.text
time.sleep(5)
with open(save_file_path, 'w') as file:
    file.write(html)

In [6]:
race_list = []
horse_list = []
soup = BeautifulSoup(html, 'html.parser')

# race基本情報
race_list.append(soup.find("span", class_="RaceNum").get_text())
race_list.append(soup.find("div", class_="RaceName").get_text())
RaceData01 = soup.find("div", class_="RaceData01").get_text()
race_list.append(RaceData01.split("/")[0])
race_list.append(RaceData01.split("/")[1])
try:
    race_list.append(RaceData01.split("/")[2])
except AttributeError:
    race_list.append(None)
try:
    race_list.append(RaceData01.split("/")[3])
except AttributeError:
    race_list.append(None)
    
RaceData02 = soup.find("div", class_="RaceData02").get_text() 
#RaceData02.split("\n")[0]#->''
#horse_list.append(RaceData02.split("\n")[1])#->'2回'
race_list.append(RaceData02.split("\n")[2])#->'中京'
#horse_list.append(RaceData02.split("\n")[3])#->'6日目'
#horse_list.append(RaceData02.split("\n")[4])#->'サラ系４歳以上'
#horse_list.append(RaceData02.split("\n")[5])#->'オープン'
#RaceData02.split("\n")[6]#->'\xa0\xa0\xa0\xa0\xa0'
#horse_list.append(RaceData02.split("\n")[7])#->'(国際)(指)'
#horse_list.append(RaceData02.split("\n")[8])#->'定量'
race_list.append(RaceData02.split("\n")[9])#->'18頭'
#RaceData02.split("\n")[10]#->''
#horse_list.append(RaceData02.split("\n")[11])#->'本賞金:17000,6800,4300,2600,1700万円'
#RaceData02.split("\n")[12]#->''

In [7]:
race_list

['\n\n11R\n',
 '高松宮記念\n\n\n\n',
 '\n15:40発走 ',
 ' 芝1200m (左)\n',
 ' 天候:小雨\n',
 ' 馬場:不\n',
 '中京',
 '18頭']

In [8]:
horse_list_list=[]

#tableの情報を取得
horse_tables = soup.findAll("table", class_="RaceTable01")
horse_table = horse_tables[0].findAll('tr', class_="HorseList")
#tableの情報から各情報を抜き出す
for i in range(len(horse_table)):
    horse_list=[]
    result_row = horse_table[i].findAll("td")
    horse_list.append(result_row[0].get_text())#枠
    horse_list.append(result_row[1].get_text())#番号
    horse_list.append(result_row[3].find('a').get('href').split("/")[-1])#馬名
    horse_list.append(result_row[4].get_text())#性別年齢
    horse_list.append(result_row[5].get_text())#斤量
    horse_list.append(result_row[6].find('a').get('href').split("/")[-2])#騎手
    horse_list.append(result_row[7].find('a').get('href').split("/")[-2])#厩舎
    horse_list.append(result_row[8].get_text())#馬体重
    horse_list.append(result_row[9].get_text())#オッズ
    horse_list.append(result_row[10].get_text())#人気
    horse_list.append(result_row[3].get_text())#馬の名前
    horse_list_list.append(horse_list)

In [9]:
horse_list_list[0]

['1',
 '1',
 '2015104364',
 '牡8',
 '58.0',
 '01091',
 '01135',
 '\n484(-12)\n',
 '---.-',
 '\n**\n',
 '\n\n\nトゥラヴェスーラ\n\n\n']

In [10]:
columns=[
        'race_id',
        'frame_number',
        'horse_number',
        'horse_id',
        'sex_and_age',
        'burden_weight',
        'rider_id',
        'tamer_id',
        'time_value',
        'tame_time',
        'owner_id'
        'horse_weight',
        'odds',
        'popular',
        'horse_name',
        ]

In [11]:
horse_df = pd.DataFrame(columns=columns)
for horse_list in horse_list_list:
    horse_se = pd.Series( horse_list, index=horse_df.columns)
    horse_df = pd.concat([horse_df, horse_se.to_frame().T], ignore_index=True)

ValueError: Length of values (11) does not match length of index (14)

In [12]:
horse_href_list=[]
for row in range(1, len(result_rows)):
    horse_href=result_rows[row].find_elements(By.TAG_NAME,"td")[3].find_element(By.TAG_NAME,"a").get_attribute("href")
    horse_href_list.append(horse_href)


HTML_HORSE_DIR = "html/"+dir+"/horsepage/"
if not os.path.isdir(HTML_HORSE_DIR):
    os.makedirs(HTML_HORSE_DIR)   

for url in horse_href_list:
    list = url.split("/")
    horse_id = list[-2]
    save_file_path = HTML_HORSE_DIR+horse_id+'.html'
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    html = response.text
    time.sleep(5)
    with open(save_file_path, 'w') as file:
        file.write(html)

NameError: name 'result_rows' is not defined